In [ ]:
!./k8s/install.sh

In [86]:
!task helm:lint CHART=rdf-delta

task: [helm:lint] helm lint charts/rdf-delta
==> Linting charts/rdf-delta

1 chart(s) linted, 0 chart(s) failed


In [119]:
!task helm:install CHART=rdf-delta

task: [helm:install] helm upgrade --install rdf-delta charts/rdf-delta --namespace rdf-delta --create-namespace --wait --timeout 10m
Release "rdf-delta" has been upgraded. Happy Helming!
NAME: rdf-delta
LAST DEPLOYED: Sat Jun 17 13:01:26 2023
NAMESPACE: rdf-delta
STATUS: deployed
REVISION: 2
NOTES:
1. Get the application URL by running these commands:
  http://rdf-delta.k8s.localdev/


In [103]:
!task helm:delete CHART=rdf-delta

task: [helm:delete] helm delete rdf-delta --namespace rdf-delta
release "rdf-delta" uninstalled


In [7]:
!kubectl create -f https://k8s.io/examples/admin/dns/busybox.yaml

pod/busybox created


In [52]:
!kubectl exec -ti busybox -- nslookup rdf-delta-0.rdf-delta-headless.rdf-delta.svc.cluster.local

Server:    10.43.0.10
Address 1: 10.43.0.10 kube-dns.kube-system.svc.cluster.local

Name:      rdf-delta-0.rdf-delta-headless.rdf-delta.svc.cluster.local
Address 1: 10.42.0.167 rdf-delta-0.rdf-delta-headless.rdf-delta.svc.cluster.local


In [37]:
!pip install SPARQLWrapper

In [105]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [106]:
sparql_endpoint = "https://rdf-delta.k8s.localdev/ds"

sparql = SPARQLWrapper(sparql_endpoint)

In [107]:
def insert_data():
    sparql_query = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX ex: <http://example.org/>

        INSERT DATA {
            ex:subject1 rdf:type ex:Person ;
                         ex:name "John" .

            ex:subject2 rdf:type ex:Person ;
                         ex:name "Jane" .
        }
    """

    sparql.setQuery(sparql_query)
    sparql.setMethod('POST')

    sparql.query()

    print("Data inserted successfully.")

In [108]:
def search_data(sparql_query=None):
    if sparql_query is None:

        sparql_query = """
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX ex: <http://example.org/>

            SELECT ?name
            WHERE {
                ?subject rdf:type ex:Person ;
                         ex:name ?name .
            }
        """

    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)

    results = sparql.query().convert()

    if 'results' in results:
        bindings = results['results']['bindings']
        if len(bindings) > 0:
            print("Results:")
            for binding in bindings:
                name = binding['name']['value']
                print("Name:", name)
        else:
            print("No results found.")
    else:
        print("No results found.")

In [109]:
def search_data_by_name(name):
    sparql_query = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX ex: <http://example.org/>

        SELECT ?name
        WHERE {
            ?subject rdf:type ex:Person ;
                     ex:name ?name .
            FILTER (str(?name) = "%s")
        }
    """ % name
    search_data(sparql_query)

In [110]:
def update_data(subject_uri, new_name):
    sparql_query = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX ex: <http://example.org/>

        DELETE {{
            <{subject_uri}> ex:name ?old_name .
        }}
        INSERT {{
            <{subject_uri}> ex:name "{new_name}" .
        }}
        WHERE {{
            <{subject_uri}> rdf:type ex:Person ;
                            ex:name ?old_name .
        }}
    """.format(subject_uri=subject_uri, new_name=new_name)

    sparql.setQuery(sparql_query)
    sparql.setMethod('POST')

    sparql.query()

    print("Data updated successfully.")

In [111]:
insert_data()

Data inserted successfully.


In [112]:
search_data()

Results:
Name: John
Name: Jane


In [113]:
search_data_by_name("John")

Results:
Name: John


In [114]:
update_data("http://example.org/subject1", "John Doe")

Data updated successfully.


In [118]:
search_data()

Results:
Name: John Doe
Name: Jane


In [116]:
search_data_by_name("John Doe")

Results:
Name: John Doe
